In [1]:
import torch
import numpy as np
import torch.utils.benchmark as benchmark
from torch import nn


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
# -------------------------------
# Bin Packing: Approximate (First Fit)
# -------------------------------
def first_fit(items, bin_capacity):
    bins_content = []
    bin_remaining_capacity = []
    valid_items = [item for item in items if 0 < item <= bin_capacity]

    for item_size in valid_items:
        placed = False
        for i in range(len(bins_content)):
            if item_size <= bin_remaining_capacity[i]:
                bins_content[i].append(item_size)
                bin_remaining_capacity[i] -= item_size
                placed = True
                break
        if not placed:
            bins_content.append([item_size])
            bin_remaining_capacity.append(bin_capacity - item_size)
    return len(bins_content)

In [7]:
# -------------------------------
# Bin Packing: Optimal (Backtracking)
# -------------------------------
min_bins_solution_global = float('inf')

def solve_optimal_recursive(items_to_pack, bin_capacity, bins_count, rem_cap):
    global min_bins_solution_global

    if not items_to_pack:
        min_bins_solution_global = min(min_bins_solution_global, bins_count)
        return

    if bins_count >= min_bins_solution_global:
        return

    item = items_to_pack[0]
    rest = items_to_pack[1:]

    for i in range(bins_count):
        if item <= rem_cap[i]:
            rem_cap[i] -= item
            solve_optimal_recursive(rest, bin_capacity, bins_count, rem_cap)
            rem_cap[i] += item

    rem_cap.append(bin_capacity - item)
    solve_optimal_recursive(rest, bin_capacity, bins_count + 1, rem_cap)
    rem_cap.pop()

def optimal_bin_packing(items, bin_capacity):
    global min_bins_solution_global
    min_bins_solution_global = float('inf')
    valid_items = sorted([item for item in items if 0 < item <= bin_capacity], reverse=True)
    if not valid_items: return 0
    solve_optimal_recursive(valid_items, bin_capacity, 0, [])
    return min_bins_solution_global



In [8]:
# -------------------------------
# PyTorch Modules
# -------------------------------
class FirstFitModule(nn.Module):
    def __init__(self, bin_capacity):
        super().__init__()
        self.bin_capacity = bin_capacity

    def forward(self, inputs: torch.Tensor):
        return torch.tensor([first_fit(x.tolist(), self.bin_capacity) for x in inputs])

class OptimalModule(nn.Module):
    def __init__(self, bin_capacity):
        super().__init__()
        self.bin_capacity = bin_capacity

    def forward(self, inputs: torch.Tensor):
        return torch.tensor([optimal_bin_packing(x.tolist(), self.bin_capacity) for x in inputs])


In [9]:
# -------------------------------
# Benchmarking Function
# -------------------------------
def benchmark_latency(model, input_tensor):
    print(f"\n🧪 Benchmarking: {model.__class__.__name__}")
    model.eval()

    # Warm-up
    model(input_tensor[:1])

    # Batch size = 2000
    t_batch = benchmark.Timer(
        stmt="model(inputs)",
        globals={"model": model, "inputs": input_tensor}
    )
    print(t_batch.timeit(5))

    # Batch size = 1
    t_single = benchmark.Timer(
        stmt="model(inputs[:1])",
        globals={"model": model, "inputs": input_tensor}
    )
    print(t_single.timeit(5))


In [10]:
# -------------------------------
# Generate Random Data
# -------------------------------
BIN_CAPACITY = 100
NUM_SAMPLES = 2000
ITEMS_PER_SAMPLE = 30

np.random.seed(42)
items_np = np.random.randint(1, BIN_CAPACITY + 1, size=(NUM_SAMPLES, ITEMS_PER_SAMPLE))
input_tensor = torch.tensor(items_np, dtype=torch.int16)

# -------------------------------
# Run Benchmarks
# -------------------------------
ff_model = FirstFitModule(bin_capacity=BIN_CAPACITY)
opt_model = OptimalModule(bin_capacity=BIN_CAPACITY)

benchmark_latency(ff_model, input_tensor)
benchmark_latency(opt_model, input_tensor)


🧪 Benchmarking: FirstFitModule
model(inputs)
  40.41 ms
  1 measurement, 5 runs , 1 thread
model(inputs[:1])
  39.83 us
  1 measurement, 5 runs , 1 thread

🧪 Benchmarking: OptimalModule
model(inputs)
  165.99 s
  1 measurement, 5 runs , 1 thread
model(inputs[:1])
  140.76 us
  1 measurement, 5 runs , 1 thread
